# Generating a WMS template

This Jupyter notebook can be used to generate a template using a WMS Server.

**Click on the cell below and click on "Run" to import the necessary libraries.**

In [1]:
from xml.etree import ElementTree
from owslib.wms import WebMapService
import requests
from collections import OrderedDict
import json

Update the WMS url in the following cell.

In [2]:
wms_url = 'https://wms.zh.ch/OGDLidarZH?SERVICE=WMS&Request=GetCapabilities'

In [3]:
def get_wms_version(url):
    response = requests.get(url)
    tree = ElementTree.fromstring(response.content)
    version = tree.attrib['version']
    return version

version = get_wms_version(wms_url)
wms = WebMapService(wms_url, version=version)

layers = list(wms.contents)

for i, layer in enumerate(layers):
    print("{}: {} ({})".format(i,
                               layers[i],
                               wms[layers[i]].title))

print(" ")
selected_layer_idx = int(input("Select Layer: "))
print(" ")

assert 0 <= selected_layer_idx < len(layers)
selected_layer = layers[selected_layer_idx]

crs_options = wms[selected_layer].crsOptions
bbox = wms[selected_layer].boundingBoxWGS84


styles = wms[selected_layer].styles
selected_style = None
if len(styles) > 0:
    for i, style in enumerate(styles):
        print("{}: {} ({})".format(i, style, styles[style]['title']))
    print(" ")
    selected_style_idx = int(input("Select Style: "))
    print(" ")
    
    assert 0 <= selected_style_idx < len(styles)
    selected_style = list(styles.keys())[selected_style_idx]
    
format_options = wms.getOperationByName('GetMap').formatOptions
for i, format_option in enumerate(format_options):
    print("{}: {}".format(i, format_option))

print(" ")
selected_format_idx = int(input("Select Format: "))
print(" ")
assert 0 <= selected_format_idx < len(format_options)
selected_format = format_options[selected_format_idx]

wms_paramters = {}
wms_paramters['LAYERS'] = selected_layer
if selected_style is not None:
    wms_paramters['STYLES'] = selected_style
wms_paramters['FORMAT'] = selected_format
wms_paramters['CRS'] = '{proj}'
wms_paramters['WIDTH'] = '{width}'
wms_paramters['HEIGHT'] = '{height}'
wms_paramters['BBOX'] = '{bbox}'
wms_paramters['VERSION'] = version
wms_paramters['SERVICE'] = 'WMS'
wms_paramters['REQUEST'] = 'GetMap'

baseurl = wms_url.split("?")[0]
service_url = baseurl + "?" + "&".join(["{}={}".format(key, value) for key, value in wms_paramters.items()])

result = OrderedDict()
result['type'] = 'Feature'
result['bbox'] = ''
result['properties'] = OrderedDict()
result['properties']['name'] = ''
result['properties']['i18n'] = False
result['properties']['type'] = 'wms'
result['properties']['url'] = service_url
result['properties']['min_zoom'] = 0
result['properties']['max_zoom'] = 1
result['properties']['permission_osm'] = ''
result['properties']['license'] = ''
result['properties']['license_url'] = ''
result['properties']['privacy_policy_url'] = ''
result['properties']['id'] = ''
result['properties']['description'] = ''
result['properties']['country_code'] = ''
result['properties']['default'] = False
result['properties']['best'] = False
result['properties']['start_date'] = ''
result['properties']['end_date'] = ''
result['properties']['no_tile_header'] = ''
result['properties']['overlay'] = ''
result['properties']['category'] = ''
result['properties']['available_projections'] = crs_options
result['properties']['attribution'] = {'url': '',
                                       'text': '',
                                       'html': '',
                                       'required': True}
result['properties']['icon'] = ''
result['geometry'] = ''

print("")
print("Copy the following output:")
print("-------------------------")
print(json.dumps(result,
                 indent=4,
                 sort_keys=False))
print("-------------------------")

0: OGDLidarZH (Digitale Höhenmodelle)
1: dtm2014hillshade (Terrainschummerung)
2: dom2014hillshade (Oberflächenschummerung)
3: blatteinteilung-lidar (Blatteinteilung Lidar)
4: blatteinteilung-dtm (Digitales Terrain Modell (DTM)/Digitales Oberflächenmodell (DOM))
5: gemeindegrenzen (Gemeindegrenzen)
6: blatteinteilung-labels_lidar (LIDAR Beschriftung)
7: blatteinteilung-labels_dtm (DTM/DOM Beschriftung)
 
Select Layer: 1
 
0: image/png
1: image/jpeg
2: image/png; mode=8bit
3: application/x-pdf
4: image/svg+xml
5: image/tiff
6: application/vnd.google-earth.kml+xml
7: application/vnd.google-earth.kmz
 
Select Format: 1
 

Copy the following output:
-------------------------
{
    "type": "Feature",
    "bbox": "",
    "properties": {
        "name": "",
        "i18n": false,
        "type": "wms",
        "url": "https://wms.zh.ch/OGDLidarZH?LAYERS=dtm2014hillshade&FORMAT=image/jpeg&CRS={proj}&WIDTH={width}&HEIGHT={height}&BBOX={bbox}&VERSION=1.3.0&SERVICE=WMS&REQUEST=GetMap",
        "m